In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import urlopen
import requests
import sqlite3
import json
import sqlite3
import os


In [2]:
#GUARDANDO LISTAS
Urls_Carrefour = []
Sellers_Carrefour = []
Country_Carrefour = []
Price_Carrefour = []
Price_Carrefour_2 = []
SKU_Carrefour = []
Title_Carrefour = []
Installment_Carrefour_quantidade = []
Installment_Carrefour_valor_parcela = []
Installment_Carrefour_valor_total = []

In [3]:
header = {'authority':'www.carrefour.com.br', 'scheme':'https', 'accept':'application/json','user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}


In [4]:
import pymysql

In [5]:

def getting_n_creating_carrefour_urls(brand):
    connection = pymysql.connect(host='mysqlserver.cnzboqhfvndh.sa-east-1.rds.amazonaws.com',
                                user='admin',
                                password='turtle316712',
                                database='Products_Brands',
                                cursorclass=pymysql.cursors.DictCursor)

    #Criando o caminho do Databae
    c = connection.cursor()

    #Criando a Query
    Sql_query = "SELECT * FROM Products WHERE Brand = '%s'" % (brand)

    #Conectando com o banco de dados
    c.execute(Sql_query)
    result = c.fetchall()
    c.close()
    connection.close()

    #Passando todos o dataframe para Lowercase
    df = pd.DataFrame()
    df['Brand'] = [item['Brand'] for item in result]
    df['Product_Name'] = [item['Name'] for item in result]

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "%20")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "%20")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "%20" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.carrefour.com.br/busca/" + df['Urls']

    return df

In [6]:
Df_products = getting_n_creating_carrefour_urls("Wacom")
Df_products

,Brand,Product_Name,Urls,Urls_search
0,Wacom,CTL472,Wacom%20CTL472,https://www.carrefour.com.br/busca/Wacom%20CTL472
1,Wacom,CTL4100,Wacom%20CTL4100,https://www.carrefour.com.br/busca/Wacom%20CTL...
2,Wacom,CTL4100WLE0,Wacom%20CTL4100WLE0,https://www.carrefour.com.br/busca/Wacom%20CTL...
3,Wacom,CTL4100WLK0,Wacom%20CTL4100WLK0,https://www.carrefour.com.br/busca/Wacom%20CTL...
4,Wacom,CTL6100WLK0,Wacom%20CTL6100WLK0,https://www.carrefour.com.br/busca/Wacom%20CTL...
5,Wacom,DTC133W0A,Wacom%20DTC133W0A,https://www.carrefour.com.br/busca/Wacom%20DTC...
6,Wacom,PTH460,Wacom%20PTH460,https://www.carrefour.com.br/busca/Wacom%20PTH460
7,Wacom,DTK1660K0A,Wacom%20DTK1660K0A,https://www.carrefour.com.br/busca/Wacom%20DTK...
8,Wacom,PTH660,Wacom%20PTH660,https://www.carrefour.com.br/busca/Wacom%20PTH660
9,Wacom,CTL6100WLE0,Wacom%20CTL6100WLE0,https://www.carrefour.com.br/busca/Wacom%20CTL...


In [21]:
Df_products['Urls_search'][0]

'https://www.carrefour.com.br/busca/Boya%20mc2'

In [17]:
urls = ['https://www.carrefour.com.br/busca/Toshiba%20HD%20externo%20canvio%20basics%201tb','https://www.carrefour.com.br/busca/Seagate%20hd%20externo%201tb']

In [7]:
def get_atributes(url):
    time.sleep(5)

    response = requests.get(url, headers=header)
    html = response.text

    bs = BeautifulSoup(html, 'html.parser')

    template = bs.find('template', attrs={'data-type':'json','data-varname':'__STATE__'})

    text = template.contents[1].string

    data = json.loads(text)

    principal_key = list(data.keys())

    keys_principal = []

    for key in principal_key:
        keys_principal.append(key)

    keys_principal = [s for s in keys_principal if 'Product' in s]
    keys_principal = [s for s in keys_principal if not '$' in s]

    Data_json = pd.DataFrame()
    Data_json['Keys'] = keys_principal
    Data_json['Code'] = Data_json['Keys'].str.partition(":")[2].str.partition(".")[0]
    Data_json = Data_json.drop_duplicates(subset=['Code'])
    Data_json['Price_id'] = "$" + Data_json['Keys'] + '.items({\"filter\":\"ALL_AVAILABLE\"}).0.sellers.0.commertialOffer'
    Data_json['Sellers_id'] = Data_json['Keys'] + '.items({\"filter\":\"ALL_AVAILABLE\"}).0.sellers.0'

    #Pegando title
    for key in Data_json['Keys']:
        try:
            Title_Carrefour.append(data[key]['productName'])
        except:
            Title_Carrefour.append("Erro")

    #Pegando SKU
    for key in Data_json['Keys']:
        try:
            SKU_Carrefour.append(data[key]['productReference'])
        except:
            SKU_Carrefour.append("Erro")

    #Pegando Url
    for key in Data_json['Keys']:
        try:
            Urls_Carrefour.append("www.carrefour.com.br" + data[key]['link'])
        except:
            Urls_Carrefour.append("Erro")

    #Pegando Price
    for key in Data_json['Price_id']:
        try:
            Price_Carrefour.append(data[key]['Price'])
        except:
            Price_Carrefour.append("Erro")

    #Pegando Price 2
    for key in Data_json['Price_id']:
        try:
            Price_Carrefour_2.append(data[key]['spotPrice'])
        except:
            Price_Carrefour_2.append("Erro")

    #Pegando Seller
    for key in Data_json['Sellers_id']:
        try:
            Sellers_Carrefour.append(data[key]['sellerName'])
        except:
            Sellers_Carrefour.append("Erro")

In [8]:
for url in tqdm(Df_products['Urls_search']):
    get_atributes(url)

100%|██████████| 10/10 [01:29<00:00,  8.94s/it]


In [24]:
get_atributes(url)

In [9]:
def creating_dataframe(urls, loja, title, price, price_2, sellers, sku):
    Dataset = pd.DataFrame()

    Dataset['Urls'] = urls
    Dataset['Loja'] = loja
    Dataset['Title'] = title
    Dataset['Price'] = price
    Dataset['Price_2'] = price_2
    Dataset['Sellers'] = sellers
    Dataset['SKU'] = sku

    return Dataset

In [10]:
Dataset_Carrefour = creating_dataframe(Urls_Carrefour,"CARREFOUR",Title_Carrefour,Price_Carrefour,Price_Carrefour_2,Sellers_Carrefour,SKU_Carrefour)

In [11]:
Dataset_Carrefour

,Urls,Loja,Title,Price,Price_2,Sellers,SKU
0,www.carrefour.com.br/mesa-digitalizadora-one-b...,CARREFOUR,Mesa Digitalizadora One By Wacom CTL472 Preta,339.9,339.9,Olist,6156460
1,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom By One - Pequena - C...,0.0,0.0,Carrefour,MP21128371W-MV21128370
2,www.carrefour.com.br/tablet-152-x-95mm-mesa-di...,CARREFOUR,Tablet - 152 x 95mm - Mesa digitalizadora One ...,0.0,0.0,Loja Waz,MP11513260
3,www.carrefour.com.br/mesa-digitalizadora-ctl47...,CARREFOUR,Mesa Digitalizadora CTL472 One By Wacom Redwood,0.0,0.0,Carrefour,MP11862600
4,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom CTL472 One By Pequen...,0.0,0.0,Carrefour,MP908133350
...,...,...,...,...,...,...,...
72,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom CTL-6100WL 4096 Pont...,1520.0,1520.0,INPOWER,MP19610271
73,www.carrefour.com.br/mesadigitalizadorawacomct...,CARREFOUR,Mesa Digitalizadora Wacom Ctl6100wle0 Intuos M...,0.0,0.0,ALLIANCEINFORMATICA COMERCIO ELETRONICO,MP922638005
74,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom Intuos Creative Medi...,1749.0,1749.0,Olist,MP928010958
75,www.carrefour.com.br/mesa-digitalizadora-wacom...,CARREFOUR,Mesa Digitalizadora Wacom Intuos Media com Blu...,0.0,0.0,Trade Hub,MP14322562


In [12]:
Dataset_Carrefour.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Software_Artemis\Data\Brands_Downloads\Wacom\Carrefour_Wacom.xlsx", index=False)